In [ ]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
print(GOOGLE_API_KEY[:6])

AIzaSy


In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vector = embeddings.embed_query("hello, world! I am Twishanu yes")
vector[:5]

[-0.027409706264734268,
 -0.010154421441257,
 0.03869868442416191,
 -0.07898290455341339,
 -0.01990700140595436]

In [16]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader

In [18]:
file_path = "D:/AI enabled Test case gen/Documents/potter.txt"  
with open(file_path, "r", encoding="utf-8") as f:
    raw_text=f.read()
# if file_path.endswith(".pdf"):
#     loader = PyPDFLoader(file_path)
# elif file_path.endswith(".docx"):
#     loader = Docx2txtLoader(file_path)
# else:
#     raise ValueError("Unsupported file type")

# documents = loader.load()

In [19]:
raw_text

'Harry Potter, an orphan known as "The Boy Who Lived," is a wizard who attends Hogwarts School of Witchcraft and Wizardry. Sorted into Gryffindor house, he befriends Ron Weasley and Hermione Granger. His main adversary is Lord Voldemort, the dark wizard who murdered his parents, James and Lily Potter.\n\nThe Harry Potter series, written by J.K. Rowling, is a globally beloved collection of seven fantasy novels chronicling the life of a young wizard, Harry Potter, and his friends Hermione Granger and Ron Weasley. The main story arc follows Harry\'s journey at Hogwarts School of Witchcraft and Wizardry and his epic struggle against the dark wizard Lord Voldemort, who murdered Harry\'s parents and seeks to conquer the wizarding world. The books are set in a hidden magical world that coexists alongside the non-magical, or "Muggle," world.\n\nThe central themes of the series are powerful and resonant, including friendship, love, sacrifice, and the eternal battle between good and evil. Each b

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [25]:
#chunking function
def chunk_text(text, chunk_size=500, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_text(text)

In [26]:
chunks = chunk_text(raw_text)

In [27]:
from langchain_community.vectorstores import Chroma

In [28]:
# vectorstore = Chroma(
#     persist_directory="./Test1_chroma_sample_doc",
#     embedding_function=embeddings
# )

# batch_size = 10 

# for i in range(0, len(chunks), batch_size):
#     batch = chunks[i:i+batch_size]
#     print(f"Processing batch {i // batch_size + 1} / {len(chunks) // batch_size + 1}")
#     vectorstore.add_documents(batch)

# print("✅ Embedding complete and saved!")
#store chunks in chroma
vectorstore = Chroma.from_texts(chunks, embedding=embeddings, persist_directory="db")

: 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash-lite",convert_system_message_to_human=True)

In [ ]:
print(model.invoke("hi").content)

Hi there! How can I help you today?


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
print(retriever)

tags=['Chroma', 'GoogleGenerativeAIEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000026B7F457FE0> search_kwargs={'k': 3}


In [ ]:
#####
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question "
    "If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

In [ ]:
## Both are fed....
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answering_chain = create_stuff_documents_chain(model, chat_prompt)

In [ ]:

rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [ ]:
response = rag_chain.invoke({"input":"what do you summarize from the given doc?"})
response["answer"]

"I need the document you're referring to in order to provide a summary. Please provide the text or a link to the document."

In [ ]:
while True:
    ask_rag("What is the name of the librarian?")